In [14]:
import pandas as pd
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
m = pd.DataFrame(['charles', 'chuck', 'charlie'], columns=["name"])
n = pd.DataFrame(['charleston', 'heston', 'chuck', 'chucky'], columns=["name"])

pd.concat([m, n])

,name
0,charles
1,chuck
2,charlie
0,charleston
1,heston
2,chuck
3,chucky


In [35]:
tfidf_vectorizer = TfidfVectorizer(token_pattern=".", ngram_range=(2, 3))
tfidf_vectorizer.fit(pd.concat([m, n])["name"])

TfidfVectorizer(ngram_range=(2, 3), token_pattern='.')

In [38]:
m["tf"] = list(tfidf_vectorizer.transform(m["name"]).toarray())
n["tf"] = list(tfidf_vectorizer.transform(n["name"]).toarray())
m.set_index("name")
n.set_index("name")

display(m)
display(n)

,name,tf,tmp
0,charles,"[0.29419054335843786, 0.29419054335843786, 0.1...",1
1,chuck,"[0.0, 0.0, 0.2636449082466762, 0.0, 0.39380438...",1
2,charlie,"[0.26358118913843614, 0.26358118913843614, 0.1...",1


,name,tf,tmp
0,charleston,"[0.22492419247755582, 0.22492419247755582, 0.1...",1
1,heston,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.27539894...",1
2,chuck,"[0.0, 0.0, 0.2636449082466762, 0.0, 0.39380438...",1
3,chucky,"[0.0, 0.0, 0.2073889253364714, 0.0, 0.30977525...",1


In [40]:
df = pd.merge(m, n, how='cross')
df['cos_sim'] = df.apply(lambda x: (1 - cosine(x["tf_x"], x["tf_y"])), axis=1)
df = df[['name_x','name_y','cos_sim']]
df

,name_x,name_y,cos_sim
0,charles,charleston,0.764553
1,charles,heston,0.081020
2,charles,chuck,0.051926
3,charles,chucky,0.040846
4,chuck,charleston,0.039700
5,chuck,heston,0.000000
6,chuck,chuck,1.000000
7,chuck,chucky,0.786622
8,charlie,charleston,0.382287
9,charlie,heston,0.000000


In [49]:
grouped = pd.DataFrame(df.set_index("name_y").groupby("name_x")["cos_sim"].nlargest(3))
display(grouped)

cos_sim
name_x  name_y              
charles charleston  0.764553
        heston      0.081020
        chuck       0.051926
charlie charleston  0.382287
        chuck       0.046524
        chucky      0.036596
chuck   chuck       1.000000
        chucky      0.786622
        charleston  0.039700